In [ ]:
import os
from pathlib import Path
import random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

from google.colab import drive

# 1) 구글 드라이브 마운트
drive.mount('/content/drive')

# 2) skeleton 이미지가 들어 있는 최상위 폴더 경로 지정
#    👉 여기만 네 환경에 맞게 수정해줘
BASE_DIR = Path("/content/drive/MyDrive/CV/Holistic_hands_frames")

print("BASE_DIR 존재:", BASE_DIR.exists())
print("하위 폴더:", [p.name for p in BASE_DIR.iterdir() if p.is_dir()])

# 3) 디바이스 + 시드 고정
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if device == "cuda":
    torch.cuda.manual_seed_all(SEED)

Mounted at /content/drive
BASE_DIR 존재: True
하위 폴더: ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'seq_embeddings_hands']
device: cpu


In [ ]:
import re

rows = []

for person_dir in BASE_DIR.iterdir():
    if not person_dir.is_dir():
        continue
    person = person_dir.name  # '1', '2', ..., '10'

    for f in sorted(person_dir.iterdir()):
        if not f.is_file():
            continue
        if not f.name.lower().endswith((".png", ".jpg", ".jpeg")):
            continue

        stem = f.stem  # 확장자 뺀 파일명

        # 1) 단어: WORD0029_검사 까지만 뽑기
        m_word = re.match(r"(WORD\d+_[^_]+)", stem)
        if m_word:
            word = m_word.group(1)   # 예: WORD0029_검사
        else:
            # 형식 안 맞는 건 스킵
            # print("WARN: word 파싱 실패:", stem)
            continue

        # 2) 각도 + 프레임 index: ..._REAL01_D_s00 처럼 되어 있음
        #    → angle = D, frame_idx = 0
        m_angle = re.search(r"_(R|D|U|F|L)_s(\d+)$", stem)
        if m_angle:
            angle = m_angle.group(1)           # 'R' / 'D' / 'U' / 'F' / 'L'
            frame_idx = int(m_angle.group(2))  # 0~15
        else:
            # 형식 안 맞으면 일단 angle=X로 묶고 frame_idx=0
            angle = "X"
            frame_idx = 0

        # 3) 시퀀스 id = 사람 / 단어 / 각도
        seq_id = f"{person}/{word}/{angle}"

        rows.append({
            "person": person,
            "word": word,         # 분류 label은 word 기준 (각도는 추가 정보)
            "angle": angle,
            "seq_id": seq_id,
            "frame_idx": frame_idx,
            "path": str(f),
        })

df_frames = pd.DataFrame(rows)
df_frames = df_frames.sort_values(["seq_id", "frame_idx"]).reset_index(drop=True)

print("총 프레임 수:", len(df_frames))
print("총 시퀀스 수:", df_frames["seq_id"].nunique())
print("단어 개수:", df_frames["word"].nunique())
print("단어 예시:", df_frames["word"].unique()[:10])

print("\nseq_id별 프레임 수 (앞 5개)")
print(df_frames.groupby("seq_id")["frame_idx"].count().head())

총 프레임 수: 17552
총 시퀀스 수: 1097
단어 개수: 22
단어 예시: ['WORD0029_검사' 'WORD0033_당뇨병' 'WORD0036_면역' 'WORD0037_감기'
 'WORD0039_변비' 'WORD0040_병명' 'WORD0041_보건소'
 'WORD0042_불면증' 'WORD0046_설사' 'WORD0062_진단서']

seq_id별 프레임 수 (앞 5개)
seq_id
1/WORD0029_검사/D    16
1/WORD0029_검사/F    16
1/WORD0029_검사/L    16
1/WORD0029_검사/R    16
1/WORD0029_검사/U    16
Name: frame_idx, dtype: int64


In [ ]:
# 1) 시퀀스 테이블
df_seq = df_frames[["seq_id", "word"]].drop_duplicates()
print("총 시퀀스 수:", len(df_seq))

# 2) 시퀀스 셔플 후 70 / 15 / 15 분할
seq_ids = df_seq["seq_id"].values
np.random.seed(SEED)
np.random.shuffle(seq_ids)

N_seq = len(seq_ids)
train_end = int(N_seq * 0.7)
val_end   = train_end + int(N_seq * 0.15)

train_seq = seq_ids[:train_end]
val_seq   = seq_ids[train_end:val_end]
test_seq  = seq_ids[val_end:]

print("train seq 수:", len(train_seq))
print("val   seq 수:", len(val_seq))
print("test  seq 수:", len(test_seq))

# 3) split 라벨 부여
seq_split_map = {}
seq_split_map.update({s: "train" for s in train_seq})
seq_split_map.update({s: "val"   for s in val_seq})
seq_split_map.update({s: "test"  for s in test_seq})

df_frames["split"] = df_frames["seq_id"].map(seq_split_map)

print("\n프레임 기준 split 분포:")
print(df_frames["split"].value_counts(normalize=True))
print(df_frames["split"].value_counts())

# 4) 단어 → 인덱스 매핑 (label2idx)
classes = sorted(df_frames["word"].unique())
label2idx = {c: i for i, c in enumerate(classes)}
idx2label = {v: k for k, v in label2idx.items()}

print("\n클래스 개수:", len(classes))
print("앞 10개 클래스:", classes[:10])

총 시퀀스 수: 1097
train seq 수: 767
val   seq 수: 164
test  seq 수: 166

프레임 기준 split 분포:
split
train    0.699180
test     0.151322
val      0.149499
Name: proportion, dtype: float64
split
train    12272
test      2656
val       2624
Name: count, dtype: int64

클래스 개수: 22
앞 10개 클래스: ['WORD0029_검사', 'WORD0033_당뇨병', 'WORD0036_면역', 'WORD0037_감기', 'WORD0039_변비', 'WORD0040_병명', 'WORD0041_보건소', 'WORD0042_불면증', 'WORD0046_설사', 'WORD0062_진단서']


In [ ]:
# 🔹 아주 단순한 transform (baseline)
base_transform = transforms.Compose([
    transforms.Resize((224, 224)),   # 전체 타일 이미지를 224×224로
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std =[0.229, 0.224, 0.225],
    ),
])

def build_tile_4x4(frame_paths):
    """
    frame_paths: 한 시퀀스에 속한 프레임 경로 리스트
    → 최대 16장만 사용해서 4×4 grid 타일 이미지 생성
    """
    grid_x, grid_y = 4, 4
    tile_w, tile_h = 96, 96
    N = grid_x * grid_y  # 16

    # 1) frame_paths에서 앞에서부터 최대 16장만 사용
    if len(frame_paths) >= N:
        paths = frame_paths[:N]
    else:
        # 부족하면 마지막 프레임으로 채워넣기
        paths = frame_paths + [frame_paths[-1]] * (N - len(frame_paths))

    imgs = []
    for p in paths:
        img = Image.open(p).convert("RGB").resize((tile_w, tile_h))
        imgs.append(img)

    # 2) 4×4로 붙이기
    canvas = Image.new("RGB", (tile_w * grid_x, tile_h * grid_y), (0,0,0))
    k = 0
    for y in range(grid_y):
        for x in range(grid_x):
            canvas.paste(imgs[k], (x * tile_w, y * tile_h))
            k += 1

    return canvas


class SkeletonTilingBaselineDataset(Dataset):
    """
    가장 단순한 baseline:
      - seq_id 단위로 묶고
      - 각 seq를 4×4 타일 1장으로 만들고
      - 그걸 CNN에 넣어서 단어 분류
    """
    def __init__(self, df_frames, split, label2idx, transform=None):
        self.df = df_frames[df_frames["split"] == split].copy()
        self.label2idx = label2idx
        self.transform = transform if transform is not None else base_transform

        self.grouped = self.df.groupby("seq_id")
        self.seq_ids = list(self.grouped.groups.keys())

    def __len__(self):
        return len(self.seq_ids)

    def __getitem__(self, idx):
        seq_id = self.seq_ids[idx]
        seq_df = self.grouped.get_group(seq_id).sort_values("frame_idx")

        frame_paths = seq_df["path"].values.tolist()
        word = seq_df["word"].iloc[0]
        label = self.label2idx[word]

        # 1개 시퀀스 → 4×4 타일 이미지 1장
        tile_img = build_tile_4x4(frame_paths)

        # 텐서로 변환
        img_tensor = self.transform(tile_img)   # [3, H, W]
        return img_tensor, torch.tensor(label, dtype=torch.long)

In [ ]:
def get_loaders(batch_size=8, num_workers=2):
    train_ds = SkeletonTilingBaselineDataset(df_frames, split="train", label2idx=label2idx)
    val_ds   = SkeletonTilingBaselineDataset(df_frames, split="val",   label2idx=label2idx)
    test_ds  = SkeletonTilingBaselineDataset(df_frames, split="test",  label2idx=label2idx)

    print("train seq:", len(train_ds), " | val seq:", len(val_ds), " | test seq:", len(test_ds))

    train_loader = DataLoader(
        train_ds, batch_size=batch_size, shuffle=True,
        num_workers=num_workers, pin_memory=(device=="cuda")
    )
    val_loader = DataLoader(
        val_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=(device=="cuda")
    )
    test_loader = DataLoader(
        test_ds, batch_size=batch_size, shuffle=False,
        num_workers=num_workers, pin_memory=(device=="cuda")
    )

    return train_loader, val_loader, test_loader


# 🔹 ResNet18 backbone 사용 (가장 무난한 baseline)
class TilingResNet18(nn.Module):
    def __init__(self, num_classes):
        super().__init__()
        base = resnet18(weights=ResNet18_Weights.IMAGENET1K_V1)
        in_features = base.fc.in_features
        base.fc = nn.Linear(in_features, num_classes)
        self.backbone = base

    def forward(self, x):
        return self.backbone(x)


def run_epoch(model, loader, criterion, optimizer=None, log_interval=20):
    if optimizer is None:
        model.eval()
    else:
        model.train()

    total_loss = 0.0
    total_correct = 0
    total_count = 0

    for batch_idx, (imgs, labels) in enumerate(loader):
        imgs = imgs.to(device)
        labels = labels.to(device)

        if optimizer is not None:
            optimizer.zero_grad()

        logits = model(imgs)
        loss = criterion(logits, labels)

        if optimizer is not None:
            loss.backward()
            optimizer.step()

        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == labels).sum().item()
        total_count += labels.size(0)

        if (optimizer is not None) and (log_interval is not None) and (batch_idx % log_interval == 0):
            print(f"  [batch {batch_idx:03d}] loss={loss.item():.4f}")

    avg_loss = total_loss / total_count
    acc = total_correct / total_count if total_count > 0 else 0.0
    return avg_loss, acc

In [ ]:
from copy import deepcopy

def train_model(num_epochs=15, lr=1e-3, weight_decay=1e-4,
                batch_size=8, log_interval=20):

    print("\n===== Tiling Baseline Training 시작 =====")

    train_loader, val_loader, test_loader = get_loaders(
        batch_size=batch_size,
        num_workers=2,
    )

    model = TilingResNet18(num_classes=len(label2idx)).to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)

    best_val_acc = 0.0
    best_state = None
    patience = 5
    no_improve = 0

    for epoch in range(1, num_epochs + 1):
        print(f"\n=== Epoch {epoch:02d} ===")
        train_loss, train_acc = run_epoch(
            model, train_loader, criterion,
            optimizer=optimizer,
            log_interval=log_interval,
        )
        val_loss, val_acc = run_epoch(
            model, val_loader, criterion,
            optimizer=None,
            log_interval=None,
        )

        print(f"[Epoch {epoch:02d}] "
              f"train_loss={train_loss:.4f}, train_acc={train_acc:.4f} | "
              f"val_loss={val_loss:.4f}, val_acc={val_acc:.4f}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_state = deepcopy(model.state_dict())
            no_improve = 0
            print(f"  -> best val acc 갱신: {best_val_acc:.4f}")
        else:
            no_improve += 1
            print(f"  (no_improve={no_improve}/{patience})")

        if no_improve >= patience:
            print(f"\nEarly stopping at epoch {epoch} (no_improve={no_improve})")
            break

    # best 모델로 test 평가
    model.load_state_dict(best_state)
    test_loss, test_acc = run_epoch(
        model, test_loader, criterion,
        optimizer=None,
        log_interval=None,
    )
    print(f"\n[Final] best_val_acc={best_val_acc:.4f}, test_acc={test_acc:.4f}")

    return {
        "best_val_acc": float(best_val_acc),
        "test_acc": float(test_acc),
    }


# 실제 실행
results = []

stats = train_model(
    num_epochs=20,
    lr=1e-3,
    weight_decay=1e-4,
    batch_size=8,
    log_interval=10,
)
results.append(stats)

results_df = pd.DataFrame(results)
print("\n===== 최종 결과 =====")
print(results_df)


===== Tiling Baseline Training 시작 =====
train seq: 767  | val seq: 164  | test seq: 166

=== Epoch 01 ===
  [batch 000] loss=3.1452
  [batch 010] loss=2.8464
  [batch 020] loss=2.1658
  [batch 030] loss=2.4295
  [batch 040] loss=2.5661
  [batch 050] loss=1.9250
  [batch 060] loss=1.6774
  [batch 070] loss=2.1742
  [batch 080] loss=2.6272
  [batch 090] loss=1.9245
[Epoch 01] train_loss=2.3716, train_acc=0.3064 | val_loss=3.6456, val_acc=0.2744
  -> best val acc 갱신: 0.2744

=== Epoch 02 ===
  [batch 000] loss=1.4403
  [batch 010] loss=0.9958
  [batch 020] loss=1.2111
  [batch 030] loss=2.1846
  [batch 040] loss=1.1000
  [batch 050] loss=1.4350
  [batch 060] loss=1.6275
  [batch 070] loss=2.6602
  [batch 080] loss=2.0586
  [batch 090] loss=1.6089
[Epoch 02] train_loss=1.7092, train_acc=0.4785 | val_loss=1.9354, val_acc=0.4268
  -> best val acc 갱신: 0.4268

=== Epoch 03 ===
  [batch 000] loss=1.3749
  [batch 010] loss=0.5502
  [batch 020] loss=1.0784
  [batch 030] loss=1.0279
  [batch 040]